In [1]:
# Import required libraries
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import accuracy_score
np.random.seed(1)

In [2]:
# Load the pre-processed data
cartier = pd.read_csv('Cartier+3-day+auctions.csv')
X_train = pd.read_csv("./cartier_train_X_price.csv")
X_test = pd.read_csv("./cartier_test_X_price.csv")
y_train = pd.read_csv("./cartier_train_y_price.csv")
y_test = pd.read_csv("./cartier_test_y_price.csv")

In [3]:
# set up parameter grids for hyperparameter tuning
lr_param_grid = {
    'penalty': ['l1', 'l2', 'elasticnet'],
    'C': [0.01, 0.1, 1, 10],
    'solver': ['liblinear', 'saga']
}

svm_param_grid = {
    'C': [0.1, 1, 10, 100],
    'kernel': ['linear', 'rbf', 'poly', 'sigmoid']
}

dt_param_grid = {
    'criterion': ['gini', 'entropy'],
    'max_depth': [2, 4, 6, 8, 10],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4]
}

In [4]:
# define performance metrics
metrics = {'accuracy': accuracy_score,
          'precision': precision_score,
           'recall': recall_score}

In [5]:
performance = pd.DataFrame({"model": [], "Accuracy": [], "Precision": [], "Recall": [], "F1": []})

In [6]:
lr_model = LogisticRegression(penalty='none', max_iter=900)
_ = lr_model.fit(X_train, np.ravel(y_train))

In [7]:
model_preds = lr_model.predict(X_test)
c_matrix = confusion_matrix(y_test, model_preds)
TP = c_matrix[1][1]
TN = c_matrix[0][0]
FP = c_matrix[0][1]
FN = c_matrix[1][0]
performance = pd.concat([performance, pd.DataFrame({'model':"default logistic", 
                                                    'Accuracy': [(TP+TN)/(TP+TN+FP+FN)], 
                                                    'Precision': [TP/(TP+FP)], 
                                                    'Recall': [TP/(TP+FN)], 
                                                    'F1': [2*TP/(2*TP+FP+FN)]
                                                     }, index=[0])])
performance

,model,Accuracy,Precision,Recall,F1
0,default logistic,1.0,1.0,1.0,1.0


In [8]:
#random search in log
score_measure = "recall"
kfolds = 5

param_grid = {
    'C':[0.1,1,10,100],
    'penalty': ['l2'],
    'solver': ['liblinear']
    
}

log_reg1 = LogisticRegression()

rand_search_log = RandomizedSearchCV(estimator = log_reg1, param_distributions=param_grid, cv=kfolds, n_iter=500,
                           scoring=score_measure, verbose=1, n_jobs=-1,  # n_jobs=-1 will utilize all available CPUs 
                           return_train_score=True)

_ = rand_search_log.fit(X_train, y_train)

print(f"The best {score_measure} score is {rand_search_log.best_score_}")
print(f"... with parameters: {rand_search_log.best_params_}")

bestRecallTree = rand_search_log.best_estimator_

C:\Users\saisi\anaconda3\lib\site-packages\sklearn\model_selection\_search.py:292: UserWarning: The total space of parameters 4 is smaller than n_iter=500. Running 4 iterations. For exhaustive searches, use GridSearchCV.
  warnings.warn(


Fitting 5 folds for each of 4 candidates, totalling 20 fits
The best recall score is 1.0
... with parameters: {'solver': 'liblinear', 'penalty': 'l2', 'C': 0.1}


C:\Users\saisi\anaconda3\lib\site-packages\sklearn\utils\validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [9]:
model_preds = rand_search_log.predict(X_test)
c_matrix = confusion_matrix(y_test, model_preds)
TP = c_matrix[1][1]
TN = c_matrix[0][0]
FP = c_matrix[0][1]
FN = c_matrix[1][0]
performance = pd.concat([performance, pd.DataFrame({'model':"rand_logl", 
                                                    'Accuracy': [(TP+TN)/(TP+TN+FP+FN)], 
                                                    'Precision': [TP/(TP+FP)], 
                                                    'Recall': [TP/(TP+FN)], 
                                                    'F1': [2*TP/(2*TP+FP+FN)]
                                                     }, index=[0])])
performance

,model,Accuracy,Precision,Recall,F1
0,default logistic,1.0,1.0,1.0,1.0
0,rand_logl,1.0,1.0,1.0,1.0


In [10]:
#grid search in log
score_measure = "recall"
kfolds = 5

param_grid = {
    'C':[0.1,1,10,100],
    'penalty': ['l2'],
    'solver': ['liblinear']
    
}

log_reg1 = LogisticRegression()
grid_search_log = GridSearchCV(log_reg1, param_grid=param_grid, cv=kfolds, 
                           scoring=score_measure, verbose=1, n_jobs=-1,  # n_jobs=-1 will utilize all available CPUs 
                           return_train_score=True)

_ = grid_search_log.fit(X_train, y_train)

print(f"The best {score_measure} score is {grid_search_log.best_score_}")
print(f"... with parameters: {grid_search_log.best_params_}")

bestRecallTree = grid_search_log.best_estimator_

Fitting 5 folds for each of 4 candidates, totalling 20 fits
The best recall score is 1.0
... with parameters: {'C': 0.1, 'penalty': 'l2', 'solver': 'liblinear'}


C:\Users\saisi\anaconda3\lib\site-packages\sklearn\utils\validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [11]:
model_preds = grid_search_log.predict(X_test)
c_matrix = confusion_matrix(y_test, model_preds)
TP = c_matrix[1][1]
TN = c_matrix[0][0]
FP = c_matrix[0][1]
FN = c_matrix[1][0]
performance = pd.concat([performance, pd.DataFrame({'model':"grid_log_l", 
                                                    'Accuracy': [(TP+TN)/(TP+TN+FP+FN)], 
                                                    'Precision': [TP/(TP+FP)], 
                                                    'Recall': [TP/(TP+FN)], 
                                                    'F1': [2*TP/(2*TP+FP+FN)]
                                                     }, index=[0])])
performance

,model,Accuracy,Precision,Recall,F1
0,default logistic,1.0,1.0,1.0,1.0
0,rand_logl,1.0,1.0,1.0,1.0
0,grid_log_l,1.0,1.0,1.0,1.0


In [12]:
#svm with linear kernel
svm_lin_model1 = SVC(kernel="linear",probability=True)
_ = svm_lin_model1.fit(X_train, np.ravel(y_train))

In [13]:
#svm with rbf kernel
svm_rbf_model1 = SVC(kernel="rbf", C=10, gamma='scale',probability=True)
_ = svm_rbf_model1.fit(X_train, np.ravel(y_train))

In [14]:
model_preds = svm_rbf_model1.predict(X_test)
c_matrix = confusion_matrix(y_test, model_preds)
TP = c_matrix[1][1]
TN = c_matrix[0][0]
FP = c_matrix[0][1]
FN = c_matrix[1][0]
performance = pd.concat([performance, pd.DataFrame({'model':"rbf_svm", 
                                                    'Accuracy': [(TP+TN)/(TP+TN+FP+FN)], 
                                                    'Precision': [TP/(TP+FP)], 
                                                    'Recall': [TP/(TP+FN)], 
                                                    'F1': [2*TP/(2*TP+FP+FN)]
                                                     }, index=[0])])
performance

,model,Accuracy,Precision,Recall,F1
0,default logistic,1.0,1.0,1.0,1.0
0,rand_logl,1.0,1.0,1.0,1.0
0,grid_log_l,1.0,1.0,1.0,1.0
0,rbf_svm,1.0,1.0,1.0,1.0


In [15]:
#svm with poly kernal
svm_poly_model1 = SVC(kernel="poly",probability=True, degree=3, coef0=1, C=10)
_ = svm_poly_model1.fit(X_train, np.ravel(y_train))

In [16]:
model_preds = svm_poly_model1.predict(X_test)
c_matrix = confusion_matrix(y_test, model_preds)
TP = c_matrix[1][1]
TN = c_matrix[0][0]
FP = c_matrix[0][1]
FN = c_matrix[1][0]
performance = pd.concat([performance, pd.DataFrame({'model':"poly_svm", 
                                                    'Accuracy': [(TP+TN)/(TP+TN+FP+FN)], 
                                                    'Precision': [TP/(TP+FP)], 
                                                    'Recall': [TP/(TP+FN)], 
                                                    'F1': [2*TP/(2*TP+FP+FN)]
                                                     }, index=[0])])
performance

,model,Accuracy,Precision,Recall,F1
0,default logistic,1.0,1.0,1.0,1.0
0,rand_logl,1.0,1.0,1.0,1.0
0,grid_log_l,1.0,1.0,1.0,1.0
0,rbf_svm,1.0,1.0,1.0,1.0
0,poly_svm,1.0,1.0,1.0,1.0


In [17]:
#random search in SVM
score_measure = "recall"
kfolds = 5

param_grid = {
    'C':[0.1,1,10,100],
    'gamma':[1,0.1,0.01,0.001],
    'kernel':['poly']
    
}

SVM_R_out = SVC()
rand_search = RandomizedSearchCV(estimator = SVM_R_out, param_distributions=param_grid, cv=kfolds, n_iter=500,
                           scoring=score_measure, verbose=1, n_jobs=-1,  # n_jobs=-1 will utilize all available CPUs 
                           return_train_score=True)

_ = rand_search.fit(X_train, y_train)

print(f"The best {score_measure} score is {rand_search.best_score_}")
print(f"... with parameters: {rand_search.best_params_}")

bestRecallTree = rand_search.best_estimator_

C:\Users\saisi\anaconda3\lib\site-packages\sklearn\model_selection\_search.py:292: UserWarning: The total space of parameters 16 is smaller than n_iter=500. Running 16 iterations. For exhaustive searches, use GridSearchCV.
  warnings.warn(


Fitting 5 folds for each of 16 candidates, totalling 80 fits
The best recall score is 1.0
... with parameters: {'kernel': 'poly', 'gamma': 1, 'C': 0.1}


C:\Users\saisi\anaconda3\lib\site-packages\sklearn\utils\validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [18]:
model_preds = rand_search.predict(X_test)
c_matrix = confusion_matrix(y_test, model_preds)
TP = c_matrix[1][1]
TN = c_matrix[0][0]
FP = c_matrix[0][1]
FN = c_matrix[1][0]
performance = pd.concat([performance, pd.DataFrame({'model':"rand_poly_svm", 
                                                    'Accuracy': [(TP+TN)/(TP+TN+FP+FN)], 
                                                    'Precision': [TP/(TP+FP)], 
                                                    'Recall': [TP/(TP+FN)], 
                                                    'F1': [2*TP/(2*TP+FP+FN)]
                                                     }, index=[0])])
performance

,model,Accuracy,Precision,Recall,F1
0,default logistic,1.0,1.0,1.0,1.0
0,rand_logl,1.0,1.0,1.0,1.0
0,grid_log_l,1.0,1.0,1.0,1.0
0,rbf_svm,1.0,1.0,1.0,1.0
0,poly_svm,1.0,1.0,1.0,1.0
0,rand_poly_svm,1.0,1.0,1.0,1.0


In [19]:
#grid search in SVM
score_measure = "recall"
kfolds = 5

param_grid = {
    'C':[0.1,1,10,100],
    'gamma':[1,0.1,0.01,0.001],
    'kernel':['poly']
    
}

SVM_G_out = SVC()
grid_search = GridSearchCV(estimator = SVM_G_out, param_grid=param_grid, cv=kfolds, 
                           scoring=score_measure, verbose=1, n_jobs=-1,  # n_jobs=-1 will utilize all available CPUs 
                           return_train_score=True)

_ = grid_search.fit(X_train, y_train)

print(f"The best {score_measure} score is {grid_search.best_score_}")
print(f"... with parameters: {grid_search.best_params_}")

bestRecallTree = grid_search.best_estimator_

Fitting 5 folds for each of 16 candidates, totalling 80 fits
The best recall score is 1.0
... with parameters: {'C': 0.1, 'gamma': 1, 'kernel': 'poly'}


C:\Users\saisi\anaconda3\lib\site-packages\sklearn\utils\validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [20]:
model_preds = grid_search.predict(X_test)
c_matrix = confusion_matrix(y_test, model_preds)
TP = c_matrix[1][1]
TN = c_matrix[0][0]
FP = c_matrix[0][1]
FN = c_matrix[1][0]
performance = pd.concat([performance, pd.DataFrame({'model':"grid_poly_svm", 
                                                    'Accuracy': [(TP+TN)/(TP+TN+FP+FN)], 
                                                    'Precision': [TP/(TP+FP)], 
                                                    'Recall': [TP/(TP+FN)], 
                                                    'F1': [2*TP/(2*TP+FP+FN)]
                                                     }, index=[0])])
performance

,model,Accuracy,Precision,Recall,F1
0,default logistic,1.0,1.0,1.0,1.0
0,rand_logl,1.0,1.0,1.0,1.0
0,grid_log_l,1.0,1.0,1.0,1.0
0,rbf_svm,1.0,1.0,1.0,1.0
0,poly_svm,1.0,1.0,1.0,1.0
0,rand_poly_svm,1.0,1.0,1.0,1.0
0,grid_poly_svm,1.0,1.0,1.0,1.0


In [21]:
#random with tree
score_measure = "recall"
kfolds = 5

param_grid = {
    'min_samples_split': np.arange(1,42),  
    'min_samples_leaf': np.arange(1,42),
    'min_impurity_decrease': np.arange(0.0001, 0.01, 0.0005),
    'max_leaf_nodes': np.arange(5, 42), 
    'max_depth': np.arange(1,42), 
    'criterion': ['entropy', 'gini'],
}

dtree = DecisionTreeClassifier()
rand_search_tree = RandomizedSearchCV(estimator = dtree, param_distributions=param_grid, cv=kfolds, n_iter=500,
                           scoring=score_measure, verbose=1, n_jobs=-1,  # n_jobs=-1 will utilize all available CPUs 
                           return_train_score=True)

_ = rand_search_tree.fit(X_train, y_train)

print(f"The best {score_measure} score is {rand_search_tree.best_score_}")
print(f"... with parameters: {rand_search_tree.best_params_}")

bestRecallTree = rand_search_tree.best_estimator_

Fitting 5 folds for each of 500 candidates, totalling 2500 fits
The best recall score is 1.0
... with parameters: {'min_samples_split': 3, 'min_samples_leaf': 20, 'min_impurity_decrease': 0.0031, 'max_leaf_nodes': 9, 'max_depth': 35, 'criterion': 'gini'}


C:\Users\saisi\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:372: FitFailedWarning: 
60 fits failed out of a total of 2500.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
60 fits failed with the following error:
Traceback (most recent call last):
  File "C:\Users\saisi\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py", line 680, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "C:\Users\saisi\anaconda3\lib\site-packages\sklearn\tree\_classes.py", line 937, in fit
    super().fit(
  File "C:\Users\saisi\anaconda3\lib\site-packages\sklearn\tree\_classes.py", line 250, in fit
    raise ValueError(
ValueError: min_samples_split must be an integer greater than 1 or a float in (

In [22]:
model_preds = rand_search_tree.predict(X_test)
c_matrix = confusion_matrix(y_test, model_preds)
TP = c_matrix[1][1]
TN = c_matrix[0][0]
FP = c_matrix[0][1]
FN = c_matrix[1][0]
performance = pd.concat([performance, pd.DataFrame({'model':"rand_poly_tree", 
                                                    'Accuracy': [(TP+TN)/(TP+TN+FP+FN)], 
                                                    'Precision': [TP/(TP+FP)], 
                                                    'Recall': [TP/(TP+FN)], 
                                                    'F1': [2*TP/(2*TP+FP+FN)]
                                                     }, index=[0])])
performance

,model,Accuracy,Precision,Recall,F1
0,default logistic,1.0,1.0,1.0,1.0
0,rand_logl,1.0,1.0,1.0,1.0
0,grid_log_l,1.0,1.0,1.0,1.0
0,rbf_svm,1.0,1.0,1.0,1.0
0,poly_svm,1.0,1.0,1.0,1.0
0,rand_poly_svm,1.0,1.0,1.0,1.0
0,grid_poly_svm,1.0,1.0,1.0,1.0
0,rand_poly_tree,1.0,1.0,1.0,1.0


In [23]:
#grid with tree
score_measure = "recall"
kfolds = 5

param_grid = {
    'min_samples_split': np.arange(3,6),  
    'min_samples_leaf': np.arange(3,6),
    'min_impurity_decrease': np.arange(0.0009, 0.0012,0.0001),
    'max_leaf_nodes': np.arange(36,40), 
    'max_depth': np.arange(39,42), 
    'criterion': ['entropy'],
}

dtree = DecisionTreeClassifier()
grid_search_tree = GridSearchCV(estimator = dtree, param_grid=param_grid, cv=kfolds, 
                           scoring=score_measure, verbose=1, n_jobs=-1,  # n_jobs=-1 will utilize all available CPUs 
                           return_train_score=True)

_ = grid_search_tree.fit(X_train, y_train)

print(f"The best {score_measure} score is {grid_search_tree.best_score_}")
print(f"... with parameters: {grid_search_tree.best_params_}")

bestRecallTree = grid_search_tree.best_estimator_

Fitting 5 folds for each of 324 candidates, totalling 1620 fits
The best recall score is 1.0
... with parameters: {'criterion': 'entropy', 'max_depth': 39, 'max_leaf_nodes': 36, 'min_impurity_decrease': 0.0009, 'min_samples_leaf': 3, 'min_samples_split': 3}


In [24]:
model_preds = grid_search_tree.predict(X_test)
c_matrix = confusion_matrix(y_test, model_preds)
TP = c_matrix[1][1]
TN = c_matrix[0][0]
FP = c_matrix[0][1]
FN = c_matrix[1][0]
performance = pd.concat([performance, pd.DataFrame({'model':"grid_poly_tree", 
                                                    'Accuracy': [(TP+TN)/(TP+TN+FP+FN)], 
                                                    'Precision': [TP/(TP+FP)], 
                                                    'Recall': [TP/(TP+FN)], 
                                                    'F1': [2*TP/(2*TP+FP+FN)]
                                                     }, index=[0])])
performance

,model,Accuracy,Precision,Recall,F1
0,default logistic,1.0,1.0,1.0,1.0
0,rand_logl,1.0,1.0,1.0,1.0
0,grid_log_l,1.0,1.0,1.0,1.0
0,rbf_svm,1.0,1.0,1.0,1.0
0,poly_svm,1.0,1.0,1.0,1.0
0,rand_poly_svm,1.0,1.0,1.0,1.0
0,grid_poly_svm,1.0,1.0,1.0,1.0
0,rand_poly_tree,1.0,1.0,1.0,1.0
0,grid_poly_tree,1.0,1.0,1.0,1.0


In [25]:
performance.sort_values(by=['Recall'])

,model,Accuracy,Precision,Recall,F1
0,default logistic,1.0,1.0,1.0,1.0
0,rand_logl,1.0,1.0,1.0,1.0
0,grid_log_l,1.0,1.0,1.0,1.0
0,rbf_svm,1.0,1.0,1.0,1.0
0,poly_svm,1.0,1.0,1.0,1.0
0,rand_poly_svm,1.0,1.0,1.0,1.0
0,grid_poly_svm,1.0,1.0,1.0,1.0
0,rand_poly_tree,1.0,1.0,1.0,1.0
0,grid_poly_tree,1.0,1.0,1.0,1.0


By the above models performed we can say that all the models are performing same. As I am getting continous variable from my data set taken, I have used thrushold value to convert to descrete.

In [26]:
%%time

ann = MLPClassifier(hidden_layer_sizes=(100,), solver='adam', max_iter=200)
_ = ann.fit(X_train, y_train)

C:\Users\saisi\anaconda3\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:1109: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


CPU times: total: 31.2 ms
Wall time: 381 ms


C:\Users\saisi\anaconda3\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:692: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(


In [27]:
model_preds = ann.predict(X_test)
c_matrix = confusion_matrix(y_test, model_preds)
TP = c_matrix[1][1]
TN = c_matrix[0][0]
FP = c_matrix[0][1]
FN = c_matrix[1][0]
performance = pd.concat([performance, pd.DataFrame({'model':"neural network-100", 
                                                    'Accuracy': [(TP+TN)/(TP+TN+FP+FN)], 
                                                    'Precision': [TP/(TP+FP)], 
                                                    'Recall': [TP/(TP+FN)], 
                                                    'F1': [2*TP/(2*TP+FP+FN)]
                                                     }, index=[0])])
performance

,model,Accuracy,Precision,Recall,F1
0,default logistic,1.0,1.0,1.0,1.0
0,rand_logl,1.0,1.0,1.0,1.0
0,grid_log_l,1.0,1.0,1.0,1.0
0,rbf_svm,1.0,1.0,1.0,1.0
0,poly_svm,1.0,1.0,1.0,1.0
0,rand_poly_svm,1.0,1.0,1.0,1.0
0,grid_poly_svm,1.0,1.0,1.0,1.0
0,rand_poly_tree,1.0,1.0,1.0,1.0
0,grid_poly_tree,1.0,1.0,1.0,1.0
0,neural network-100,1.0,1.0,1.0,1.0


In [28]:
performance.sort_values(by='Recall')

,model,Accuracy,Precision,Recall,F1
0,default logistic,1.0,1.0,1.0,1.0
0,rand_logl,1.0,1.0,1.0,1.0
0,grid_log_l,1.0,1.0,1.0,1.0
0,rbf_svm,1.0,1.0,1.0,1.0
0,poly_svm,1.0,1.0,1.0,1.0
0,rand_poly_svm,1.0,1.0,1.0,1.0
0,grid_poly_svm,1.0,1.0,1.0,1.0
0,rand_poly_tree,1.0,1.0,1.0,1.0
0,grid_poly_tree,1.0,1.0,1.0,1.0
0,neural network-100,1.0,1.0,1.0,1.0


By the above models we can say that all the models are overfit, so that we are getting 1's as classiffication metrics.

Deep Network

In [29]:
import tensorflow as tf
from tensorflow import keras
# fix random seed for reproducibility
np.random.seed(1)
tf.random.set_seed(1)

In [30]:
%%time

# create model stucture
model = keras.models.Sequential()
model.add(keras.layers.Input(7))
model.add(keras.layers.Dense(50, activation='relu'))
model.add(keras.layers.Dense(50, activation='relu'))
model.add(keras.layers.Dense(50, activation='relu'))
model.add(keras.layers.Dense(10, activation='softmax')) # final layer, 10 categories


# compile
model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# if you want to overide the defaults for the optimizer....
#adam = keras.optimizers.Adam(learning_rate=0.01)
#model.compile(loss='sparse_categorical_crossentropy', optimizer=adam, metrics=['accuracy'])

CPU times: total: 46.9 ms
Wall time: 148 ms


In [31]:
%%time

# fit the model
history = model.fit(X_train, y_train, 
                    validation_data=(X_test, y_test), 
                    epochs=20, batch_size=100)


Epoch 1/20
2/2 [==============================] - 1s 211ms/step - loss: 2.3148 - accuracy: 0.0200 - val_loss: 2.2176 - val_accuracy: 0.2100
Epoch 2/20
2/2 [==============================] - 0s 39ms/step - loss: 2.1940 - accuracy: 0.2667 - val_loss: 2.1039 - val_accuracy: 0.5600
Epoch 3/20
2/2 [==============================] - 0s 36ms/step - loss: 2.0798 - accuracy: 0.6867 - val_loss: 1.9926 - val_accuracy: 0.7700
Epoch 4/20
2/2 [==============================] - 0s 46ms/step - loss: 1.9685 - accuracy: 0.8800 - val_loss: 1.8819 - val_accuracy: 0.8700
Epoch 5/20
2/2 [==============================] - 0s 33ms/step - loss: 1.8574 - accuracy: 0.8933 - val_loss: 1.7706 - val_accuracy: 0.8900
Epoch 6/20
2/2 [==============================] - 0s 35ms/step - loss: 1.7431 - accuracy: 0.9333 - val_loss: 1.6588 - val_accuracy: 0.8900
Epoch 7/20
2/2 [==============================] - 0s 40ms/step - loss: 1.6284 - accuracy: 0.9333 - val_loss: 1.5453 - val_accuracy: 0.8900
Epoch 8/20
2/2 [==========

In [32]:
# evaluate the model

scores = model.evaluate(X_test, y_test, verbose=0)
scores
# In results, first is loss, second is accuracy

[0.3206002712249756, 0.9900000095367432]

In [33]:
# let's format this into a better output...

print("%s: %.2f" % (model.metrics_names[0], scores[0]))
print("%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))

loss: 0.32
accuracy: 99.00%


Wide and Deep Network

In [34]:
#Define the model: for multi-class

model = keras.models.Sequential()

model.add(keras.layers.Input(shape=7))
model.add(keras.layers.Dense(100, activation='relu'))
model.add(keras.layers.Dense(100, activation='relu'))
model.add(keras.layers.Dense(100, activation='relu'))
model.add(keras.layers.Dense(10, activation='softmax'))

In [35]:
# Compile model

#Optimizer:
adam = keras.optimizers.Adam(learning_rate=0.01)
model.compile(loss='sparse_categorical_crossentropy', optimizer=adam, metrics=['accuracy'])

In [36]:
# Fit the model

history = model.fit(X_train, y_train, 
                    validation_data=(X_test, y_test), 
                    epochs=20, batch_size=100)

Epoch 1/20
2/2 [==============================] - 1s 167ms/step - loss: 2.2240 - accuracy: 0.3267 - val_loss: 0.9205 - val_accuracy: 0.9800
Epoch 2/20
2/2 [==============================] - 0s 47ms/step - loss: 0.7561 - accuracy: 0.9667 - val_loss: 0.2001 - val_accuracy: 0.9200
Epoch 3/20
2/2 [==============================] - 0s 34ms/step - loss: 0.1170 - accuracy: 0.9667 - val_loss: 0.0348 - val_accuracy: 0.9900
Epoch 4/20
2/2 [==============================] - 0s 37ms/step - loss: 0.0237 - accuracy: 1.0000 - val_loss: 0.0153 - val_accuracy: 0.9900
Epoch 5/20
2/2 [==============================] - 0s 42ms/step - loss: 0.0039 - accuracy: 1.0000 - val_loss: 0.0109 - val_accuracy: 1.0000
Epoch 6/20
2/2 [==============================] - 0s 40ms/step - loss: 0.0028 - accuracy: 1.0000 - val_loss: 0.0072 - val_accuracy: 1.0000
Epoch 7/20
2/2 [==============================] - 0s 39ms/step - loss: 1.9796e-04 - accuracy: 1.0000 - val_loss: 0.0140 - val_accuracy: 0.9900
Epoch 8/20
2/2 [======

In [37]:
# evaluate the model

scores = model.evaluate(X_test, y_test, verbose=0)
scores

# In results, first is loss, second is accuracy

[0.03416389599442482, 0.9900000095367432]

In [38]:
# extract the accuracy from model.evaluate

print("%s: %.2f" % (model.metrics_names[0], scores[0]))
print("%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))

loss: 0.03
accuracy: 99.00%


In [39]:
X_train.shape

(150, 7)

In [40]:
y_test.value_counts()

Price_cat
0            69
1            31
dtype: int64

RandomGridSearch - normal

In [41]:
%%time

# If you don't have the following installed, from command line '!pip install scikeras'
from scikeras.wrappers import KerasClassifier
from keras.initializers import GlorotNormal

score_measure = "accuracy"
kfolds = 5

def build_clf(hidden_layer_sizes, dropout):
    ann = tf.keras.models.Sequential()
    ann.add(keras.layers.Input(shape=7)),
    for hidden_layer_size in hidden_layer_sizes:
        model.add(keras.layers.Dense(hidden_layer_size, kernel_initializer= tf.keras.initializers.GlorotNormal(), 
                                     bias_initializer=keras.initializers.RandomNormal(mean=0.0, stddev=0.05, seed=None), activation="relu"))
        model.add(keras.layers.Dropout(dropout))
    ann.add(tf.keras.layers.Dense(1, activation='sigmoid'))
    ann.compile(loss = 'binary_crossentropy', metrics = ['accuracy'])
    return ann

CPU times: total: 0 ns
Wall time: 38.2 ms


In [42]:
from scikeras.wrappers import KerasClassifier

keras_clf = KerasClassifier(
    model=build_clf,
    hidden_layer_sizes=7,
    dropout = 0.0
)

In [43]:
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.model_selection import RandomizedSearchCV

params = {
    'optimizer__learning_rate': [0.0005, 0.001, 0.005],
    'model__hidden_layer_sizes': [(1000,)],
    'model__dropout': [0, 0.1],
    'batch_size':[20, 60, 100],
    'epochs':[10],
    'optimizer':["adam"]
}
keras_clf.get_params().keys()


dict_keys(['model', 'build_fn', 'warm_start', 'random_state', 'optimizer', 'loss', 'metrics', 'batch_size', 'validation_batch_size', 'verbose', 'callbacks', 'validation_split', 'shuffle', 'run_eagerly', 'epochs', 'hidden_layer_sizes', 'dropout', 'class_weight'])

In [44]:
rnd_search_cv = RandomizedSearchCV(estimator=keras_clf, param_distributions=params, scoring='accuracy', n_iter=18, cv=5)

import sys
sys.setrecursionlimit(10000) # note: the default is 3000 (python 3.9)

earlystop = EarlyStopping(monitor='val_loss', patience=5, verbose=0, mode='auto')
callback = [earlystop]

_ = rnd_search_cv.fit(X_train, y_train, callbacks=callback, verbose=0)


1/1 [==============================] - 0s 47ms/step


In [45]:
best_net = rnd_search_cv.best_estimator_
print(rnd_search_cv.best_params_)

{'optimizer__learning_rate': 0.001, 'optimizer': 'adam', 'model__hidden_layer_sizes': (1000,), 'model__dropout': 0.1, 'epochs': 10, 'batch_size': 20}


In [46]:
model_preds = rnd_search_cv.predict(X_test)
c_matrix = confusion_matrix(y_test, model_preds)
TP = c_matrix[1][1]
TN = c_matrix[0][0]
FP = c_matrix[0][1]
FN = c_matrix[1][0]
performance = pd.concat([performance, pd.DataFrame({'model':"rand_DNN_NORMAL", 
                                                    'Accuracy': [(TP+TN)/(TP+TN+FP+FN)], 
                                                    'Precision': [TP/(TP+FP)], 
                                                    'Recall': [TP/(TP+FN)], 
                                                    'F1': [2*TP/(2*TP+FP+FN)]
                                                     }, index=[0])])
performance

5/5 [==============================] - 0s 4ms/step


,model,Accuracy,Precision,Recall,F1
0,default logistic,1.00,1.000000,1.000000,1.000000
0,rand_logl,1.00,1.000000,1.000000,1.000000
0,grid_log_l,1.00,1.000000,1.000000,1.000000
0,rbf_svm,1.00,1.000000,1.000000,1.000000
0,poly_svm,1.00,1.000000,1.000000,1.000000
0,rand_poly_svm,1.00,1.000000,1.000000,1.000000
0,grid_poly_svm,1.00,1.000000,1.000000,1.000000
0,rand_poly_tree,1.00,1.000000,1.000000,1.000000
0,grid_poly_tree,1.00,1.000000,1.000000,1.000000
0,neural network-100,1.00,1.000000,1.000000,1.000000


In the above model, adding DNN gives the acuracy score of 0.58 even though it is overfitting and remaining models, I am getting the acuracy score of 1 because of the model got overfitted.